# Ingestão de Dados - MySQL

Este notebook realiza a ingestão de dados de um banco de dados MySQL para o MinIO usando DeltaLake.

## Configuração

Configure as variáveis abaixo antes de executar:

In [ ]:
# Importar configurações base
%run ../00_configuracao_inicial.ipynb

In [ ]:
# ============================================
# CONFIGURAÇÕES DE CONEXÃO MYSQL
# ============================================
import os

# Configurações de conexão MySQL
MYSQL_HOST = os.getenv('MYSQL_HOST', 'localhost')
MYSQL_PORT = os.getenv('MYSQL_PORT', '3306')
MYSQL_DATABASE = os.getenv('MYSQL_DATABASE', 'mysql')
MYSQL_USER = os.getenv('MYSQL_USER', 'root')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD', 'senha')

# Configurações de leitura
MYSQL_TABLE = os.getenv('MYSQL_TABLE', 'nome_tabela')

# Configurações de destino no MinIO
DESTINO_BRONZE = f"{PATH_BRONZE}/mysql/{MYSQL_DATABASE.lower()}/{MYSQL_TABLE.lower()}"

print("Configurações MySQL:")
print(f"Host: {MYSQL_HOST}")
print(f"Port: {MYSQL_PORT}")
print(f"Database: {MYSQL_DATABASE}")
print(f"Table: {MYSQL_TABLE}")
print(f"Destino: {DESTINO_BRONZE}")

In [ ]:
# Instalar driver MySQL (executar apenas uma vez)
# !pip install pymysql mysql-connector-python

# O Spark já inclui o driver JDBC MySQL por padrão (Connector/J)

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# URL de conexão JDBC MySQL
jdbc_url = f"jdbc:mysql://{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}?useSSL=false&serverTimezone=UTC"

# Propriedades de conexão
connection_properties = {
    "user": MYSQL_USER,
    "password": MYSQL_PASSWORD,
    "driver": "com.mysql.cj.jdbc.Driver"  # MySQL 8+
    # Para MySQL 5.x usar: "com.mysql.jdbc.Driver"
}

print(f"JDBC URL: {jdbc_url}")

In [ ]:
# Função para ler dados do MySQL
def ler_mysql_table(table_name, database=None, query=None, partition_column=None, num_partitions=None, lower_bound=None, upper_bound=None):
    """
    Lê dados de uma tabela MySQL
    
    Args:
        table_name: Nome da tabela
        database: Nome do banco (opcional, usa o da conexão se não informado)
        query: Query SQL customizada (opcional, substitui table_name)
        partition_column: Coluna para particionamento paralelo (opcional)
        num_partitions: Número de partições (opcional)
        lower_bound: Valor mínimo para particionamento (opcional)
        upper_bound: Valor máximo para particionamento (opcional)
    
    Returns:
        DataFrame do Spark
    """
    if query:
        # Usar query customizada (subquery)
        table_or_query = f"({query}) mysql_table"
    elif database:
        table_or_query = f"{database}.{table_name}"
    else:
        table_or_query = table_name
    
    reader = spark.read.format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", table_or_query) \
        .option("user", MYSQL_USER) \
        .option("password", MYSQL_PASSWORD) \
        .option("driver", "com.mysql.cj.jdbc.Driver")
    
    # Adicionar opções de particionamento se fornecidas
    if partition_column and num_partitions:
        reader = reader.option("partitionColumn", partition_column) \
                      .option("numPartitions", num_partitions)
        if lower_bound is not None and upper_bound is not None:
            reader = reader.option("lowerBound", lower_bound) \
                          .option("upperBound", upper_bound)
    
    df = reader.load()
    
    return df

In [ ]:
# Exemplo 1: Leitura simples de tabela
print("Exemplo 1: Leitura simples")
df_mysql = ler_mysql_table(
    table_name=MYSQL_TABLE,
    database=MYSQL_DATABASE
)

print(f"Total de registros: {df_mysql.count()}")
df_mysql.printSchema()
df_mysql.show(5, truncate=False)

In [ ]:
# Exemplo 2: Leitura com query customizada
print("Exemplo 2: Leitura com query customizada")
query_customizada = f"""
    SELECT 
        coluna1,
        coluna2,
        coluna3,
        updated_at
    FROM {MYSQL_DATABASE}.{MYSQL_TABLE}
    WHERE updated_at >= DATE_SUB(NOW(), INTERVAL 30 DAY)
    ORDER BY updated_at DESC
"""

# df_mysql_query = ler_mysql_table(query=query_customizada)
# df_mysql_query.show(5)

In [ ]:
# Exemplo 3: Leitura com particionamento paralelo (para tabelas grandes)
print("Exemplo 3: Leitura com particionamento")
# df_mysql_partitioned = ler_mysql_table(
#     table_name=MYSQL_TABLE,
#     database=MYSQL_DATABASE,
#     partition_column="id",  # Coluna numérica para particionamento
#     num_partitions=10,
#     lower_bound=1,
#     upper_bound=1000000
# )
# df_mysql_partitioned.show(5)

In [ ]:
# Adicionar metadados de ingestão
df_ingestao = df_mysql \
    .withColumn("fonte", lit("MYSQL")) \
    .withColumn("database_origem", lit(MYSQL_DATABASE)) \
    .withColumn("tabela_origem", lit(MYSQL_TABLE)) \
    .withColumn("ingestao_em", current_timestamp()) \
    .withColumn("particao_data", date_format(current_date(), "yyyy-MM-dd"))

print("Metadados adicionados:")
df_ingestao.select("fonte", "database_origem", "tabela_origem", "ingestao_em").show(1, truncate=False)

In [ ]:
# Salvar no MinIO como Delta Table
print(f"Salvando dados em: {DESTINO_BRONZE}")

# save_delta_table(
#     df_ingestao,
#     DESTINO_BRONZE,
#     mode="overwrite",  # ou "append" para incrementais
#     partition_by=["particao_data"]  # Particionar por data
# )

print("Ingestão concluída com sucesso!")

In [ ]:
# Verificar dados salvos
# df_verificacao = read_delta_table(DESTINO_BRONZE)
# print(f"Registros salvos: {df_verificacao.count()}")
# df_verificacao.show(5)

## Ingestão Incremental

Para ingestões incrementais baseadas em timestamp ou ID:

In [ ]:
# Função para ingestão incremental
def ingestao_incremental_mysql(table_name, database, coluna_timestamp="updated_at", ultima_execucao=None):
    """
    Realiza ingestão incremental de dados MySQL
    
    Args:
        table_name: Nome da tabela
        database: Nome do banco
        coluna_timestamp: Nome da coluna de timestamp para filtro
        ultima_execucao: Timestamp da última execução (formato: 'YYYY-MM-DD HH:MI:SS')
    """
    if ultima_execucao:
        query = f"""
            SELECT * FROM {database}.{table_name}
            WHERE {coluna_timestamp} > '{ultima_execucao}'
            ORDER BY {coluna_timestamp}
        """
    else:
        # Primeira execução: pegar últimos 7 dias
        query = f"""
            SELECT * FROM {database}.{table_name}
            WHERE {coluna_timestamp} >= DATE_SUB(NOW(), INTERVAL 7 DAY)
            ORDER BY {coluna_timestamp}
        """
    
    df_incremental = ler_mysql_table(query=query)
    
    return df_incremental

# Exemplo de uso
# df_incremental = ingestao_incremental_mysql(
#     table_name=MYSQL_TABLE,
#     database=MYSQL_DATABASE,
#     ultima_execucao="2024-01-01 00:00:00"
# )
# 
# # Salvar em modo append
# save_delta_table(df_incremental, DESTINO_BRONZE, mode="append", partition_by=["particao_data"])